In [34]:
import os
import sys

curr_dir = os.getcwd()
src_dir = os.path.join(os.path.dirname(curr_dir), "src")
sql_dir = os.path.join(os.path.dirname(curr_dir), "sql")
sys.path.append(src_dir)
sys.path.append(sql_dir)

In [35]:
import warnings
from ast import literal_eval

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from db import get_db
from settings import Path
from sqlalchemy import text

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 100)

### Data Extraction from SQL Database

In [36]:
def fetch_data(source: str):
    """
    Fetches data from a specified source and returns it as a pandas DataFrame.

    Parameters:
    source (str): The name of the source file containing the SQL query.

    Returns:
    pandas.DataFrame: The fetched data as a DataFrame.

    """
    db = get_db()

    with open(os.path.join(sql_dir, source), "r") as f:
        query = text(f.read())
    result = db.execute(query)
    data = result.fetchall()
    columns = result.keys()
    df = pd.DataFrame(data, columns=columns)

    db.close()

    return df

In [37]:
steam_data = fetch_data("get_all_steam_data.sql")
steamspy_data = fetch_data("get_all_steamspy_data.sql")

Removing appid's that are not present in `steam_data` in `steamspy_data`. Reason is these games are dicontinued or do not exist anymore.

In [38]:
appids_to_remove = steamspy_data[~steamspy_data['appid'].isin(steam_data['appid'])]['appid']
steamspy_data = steamspy_data[~steamspy_data['appid'].isin(appids_to_remove)]
steamspy_data

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,languages,genre,tags
0,10,Counter-Strike,Valve,Valve,,231805,6061,0.0,"10,000,000 .. 20,000,000",0,0,0,0,999.0,999.0,0,12571,"English, French, German, Italian, Spanish - Sp...",Action,"{""FPS"": 4899, ""PvP"": 907, ""1980s"": 278, ""1990'..."
1,20,Team Fortress Classic,Valve,Valve,,7136,1087,0.0,"50,000 .. 100,000",0,0,0,0,499.0,499.0,0,84,"English, French, German, Italian, Spanish - Sp...",Action,"{""FPS"": 327, ""Mod"": 39, ""Co-op"": 98, ""Funny"": ..."
2,30,Day of Defeat,Valve,Valve,,6140,668,0.0,"5,000,000 .. 10,000,000",0,0,0,0,499.0,499.0,0,88,"English, French, German, Italian, Spanish - Spain",Action,"{""FPS"": 798, ""War"": 158, ""Co-op"": 36, ""Retro"":..."
3,40,Deathmatch Classic,Valve,Valve,,2457,518,0.0,"100,000 .. 200,000",0,0,0,0,499.0,499.0,0,4,"English, French, German, Italian, Spanish - Sp...",Action,"{""FPS"": 150, ""Gore"": 20, ""Co-op"": 16, ""Retro"":..."
4,50,Half-Life: Opposing Force,Gearbox Software,Valve,,21262,1086,0.0,"2,000,000 .. 5,000,000",0,0,0,0,499.0,499.0,0,109,"English, French, German, Korean",Action,"{""FPS"": 917, ""Gore"": 57, ""Co-op"": 43, ""Retro"":..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73498,3031590,13F,PixelCat,PixelCat,,2,0,0.0,"0 .. 20,000",0,0,0,0,239.0,399.0,40,1,"English, Japanese, Simplified Chinese, Traditi...","Adventure, Indie, RPG, Simulation","{""2D"": 96, ""RPG"": 169, ""Gore"": 20, ""Indie"": 20..."
73499,3032530,Grocery Simulator,Roman Savenok,Roman Savenok,,0,2,0.0,"0 .. 20,000",0,0,0,0,594.0,699.0,15,0,English,"Action, Indie, Simulation, Early Access","{""3D"": 28, ""Indie"": 21, ""Action"": 21, ""Combat""..."
73500,3033880,Quescaper,Neki4 Electronics,Neki4 Electronics,,2,0,0.0,"0 .. 20,000",0,0,0,0,121.0,199.0,39,0,"English, Russian","Action, Adventure, Casual","{""3D"": 107, ""Cute"": 97, ""Action"": 142, ""Arcade..."
73501,3035710,USD Clicker,RazDva Games,RazDva Games,,1,0,0.0,"0 .. 20,000",0,0,0,0,79.0,99.0,20,0,English,"Casual, Indie, Simulation","{""Idler"": 499, ""America"": 429, ""Clicker"": 505,..."


In [39]:
steam_data

,type,name,appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,supported_languages,reviews,header_image,capsule_image,website,pc_requirements,developers,publishers,pc_platform,metacritic,categories,genres,recommendations,achievements,release_date,coming_soon
0,game,Counter-Strike,10,0,0,None,[],Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,"English\n*\n, French\n*\n, German\n*\n, Italia...",None,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,,"{""minimum"": {""Minimum:"": ""500 mhz processor, 9...","[""Valve""]","[""Valve""]",1,88,"[{""id"": 1, ""description"": ""Multi-player""}, {""i...","[{""id"": ""1"", ""description"": ""Action""}]",150194,0,2000-11-01,0
1,game,Team Fortress Classic,20,0,0,None,[],One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,"English, French, German, Italian, Spanish - Sp...",None,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,,"{""minimum"": {""Minimum:"": ""500 mhz processor, 9...","[""Valve""]","[""Valve""]",1,0,"[{""id"": 1, ""description"": ""Multi-player""}, {""i...","[{""id"": ""1"", ""description"": ""Action""}]",6116,0,1999-04-01,0
2,game,Day of Defeat,30,0,0,None,[],Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,"English, French, German, Italian, Spanish - Spain",None,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,http://www.dayofdefeat.com/,"{""minimum"": {""Minimum:"": ""500 mhz processor, 9...","[""Valve""]","[""Valve""]",1,79,"[{""id"": 1, ""description"": ""Multi-player""}, {""i...","[{""id"": ""1"", ""description"": ""Action""}]",4066,0,2003-05-01,0
3,game,Deathmatch Classic,40,0,0,None,[],Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,"English, French, German, Italian, Spanish - Sp...",None,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,,"{""minimum"": {""Minimum:"": ""500 mhz processor, 9...","[""Valve""]","[""Valve""]",1,0,"[{""id"": 1, ""description"": ""Multi-player""}, {""i...","[{""id"": ""1"", ""description"": ""Action""}]",2137,0,2001-06-01,0
4,game,Half-Life: Opposing Force,50,0,0,None,[],Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,"English, French, German, Korean",None,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,,"{""minimum"": {""Minimum:"": ""500 mhz processor, 9...","[""Gearbox Software""]","[""Valve""]",1,0,"[{""id"": 2, ""description"": ""Single-player""}, {""...","[{""id"": ""1"", ""description"": ""Action""}]",19180,0,1999-11-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73365,game,13F,3031590,0,0,full,[],You are a tired office worker who works overti...,You are a tired office worker who works overti...,You are a tired office worker who has been wor...,"English\n*\n, Japanese\n*\n, Simplified Chines...",None,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,,"{""minimum"": {""1 GB RAM"": ""Graphics:"", ""Intel i...","[""PixelCat""]","[""PixelCat""]",1,0,"[{""id"": 2, ""description"": ""Single-player""}, {""...","[{""id"": ""25"", ""description"": ""Adventure""}, {""i...",0,7,2024-06-20,0
73366,game,Grocery Simulator,3032530,0,0,None,[],"Realistic Store Management\nDesign, customize,...","Realistic Store Management\nDesign, customize,

In [40]:
null_counts = steam_data.isnull().sum()
null_counts

type                        0
name                        0
appid                       0
required_age                0
is_free                     0
controller_support      56711
dlc                         0
detailed_description       70
about_the_game             72
short_description          63
supported_languages        82
reviews                 63079
header_image                0
capsule_image               0
website                     0
pc_requirements             0
developers                  0
publishers                  0
pc_platform                 0
metacritic                  0
categories                  0
genres                      0
recommendations             0
achievements                0
release_date              212
coming_soon                 0
dtype: int64

In [41]:
null_counts = steamspy_data.isnull().sum()
null_counts

appid              0
name               0
developer          0
publisher          0
score_rank         0
positive           0
negative           0
userscore          0
owners             0
average_forever    0
average_2weeks     0
median_forever     0
median_2weeks      0
price              1
initialprice       1
discount           1
ccu                0
languages          1
genre              0
tags               0
dtype: int64

### Processing Age

In [42]:
steam_data['required_age'].value_counts()

required_age
0     72569
17      326
18      236
16       60
13       56
12       48
15       35
7        10
10        7
6         6
14        6
3         6
11        4
5         1
Name: count, dtype: int64

Reducing the number of categories that ages fall into by following the [PEGI age ratings](https://pegi.info) categories. For example, instead of comparing games rated as 5, 6, 7 or 8, we could compare games rated 5+ or 8+.

In [46]:
def process_age(df):
    """
    Process the age column in the given DataFrame by converting the age ratings to specific age groups.

    Parameters:
    df (DataFrame): The DataFrame containing the age column.

    Returns:
    DataFrame: The DataFrame with the age column processed.
    """
    # PEGI Age ratings: 3, 7, 12, 16, 18
    cut_points = [-1, 0, 3, 7, 12, 16, 18]
    categories = [0, 3, 7, 12, 16, 18]
    
    df['required_age'] = pd.cut(df['required_age'], bins=cut_points, labels=categories)
    
    return df

In [47]:
age_df = process_age(steam_data)
age_df['required_age'].value_counts().sort_index()

required_age
0     72569
3         6
7        17
12       59
16      157
18      562
Name: count, dtype: int64

In [54]:
steam_data['coming_soon'].value_counts()


coming_soon
0    73264
1      106
Name: count, dtype: int64

In [58]:
steam_data['release_date'].isna().value_counts()

release_date
False    73158
True       212
Name: count, dtype: int64

In [64]:
steam_data[steam_data['release_date'].isna() & steam_data['coming_soon']==1]

,type,name,appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,supported_languages,reviews,header_image,capsule_image,website,pc_requirements,developers,publishers,pc_platform,metacritic,categories,genres,recommendations,achievements,release_date,coming_soon
1907,game,ASYLUM,230210,0,0,full,[],From the creators of cult sleeper hit\nScratch...,From the creators of cult sleeper hit\nScratch...,An epic supernatural horror adventure and the ...,"English\n*\n, French, Italian, German, Spanish...",None,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,http://senscape.io,"{""minimum"": {""4 GB RAM"": ""Graphics:"", ""Minimum...","[""Senscape""]","[""Senscape""]",1,0,"[{""id"": 2, ""description"": ""Single-player""}, {""...","[{""id"": ""25"", ""description"": ""Adventure""}, {""i...",0,0,NaT,1
3083,game,Future Perfect,282370,0,0,None,[],Development Updates\nTo receive updates on Fut...,Future Perfect breaks down the barrier between...,Future Perfect breaks down the barrier between...,English,None,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,http://futureperfectgame.com,"{""minimum"": {""Minimum:"": ""Additional Notes:""},...","[""Unknown Worlds Entertainment""]","[""Unknown Worlds Entertainment ""]",1,0,"[{""id"": 1, ""description"": ""Multi-player""}, {""i...","[{""id"": ""1"", ""description"": ""Action""}, {""id"": ...",0,0,NaT,1
4704,game,Don't Move,334350,0,0,full,[],"Don't Move is about ninjas, failure, ludonarra...","Don't Move is about ninjas, failure, ludonarra...",Don't Move is an experimental puzzle platforme...,"English, French, German, Spanish - Spain, Bulg...",“Don’t Move Distills Modern Gaming’s 'Player I...,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,http://dontmove.co/,"{""minimum"": {""2 GB RAM"": ""Graphics:"", ""Minimum...","[""STVR""]","[""STVR""]",1,0,"[{""id"": 2, ""description"": ""Single-player""}, {""...","[{""id"": ""1"", ""description"": ""Action""}, {""id"": ...",0,0,NaT,1
5220,game,Diamond,346390,0,0,full,[],Feature List\nCreate and share levels using th...,Diamond is a fast paced puzzle game demanding ...,Diamond is a fast paced puzzle game where you ...,English,None,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,http://www.officialdiamondgame.com/,"{""minimum"": {""1 GB RAM"": ""Graphics:"", ""Minimum...","[""Brandon Cassata""]","[""None""]",1,0,"[{""id"": 2, ""description"": ""Single-player""}, {""...","[{""id"": ""4"", ""description"": ""Casual""}, {""id"": ...",0,0,NaT,1
5882,game,Cadence,362800,0,0,None,[],Cadence is something different: a hybrid of ga...,Cadence is something different: a hybrid of ga...,Relax. Solve Problems. Make Music. Feel Nouris...,English,None,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,,"{""minimum"": {""2 GHz"": ""Memory:"", ""Minimum:"": ""...","[""Made With Monster Love""]","[""Made With Monster Love""]",1,0,"[{""id"": 2, ""description"": ""Single-player""}, {""...","[{""id"": ""23"", ""description"": ""Indie""}]",0,0,NaT,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25588,game,Tano's Fate,888910,0,0,full,[],An unknown arrival. A land disconnected from t...,An unknown arrival. A land disconnected from t...,An unknown arrival. A land disconnected from t...,English\n*\n*\nlanguages with full audio support,None,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,https://tanosfate.com,"{""minimum"": {""OS *:"": ""Windows 7/8.1/10 (64 Bi...","[""Jay-Cast Studios""]","[""Jay-Cast Studios""]",1,0,"[{""id"": 2, ""description"": ""Single-player""}, {""...","[{""id"": ""25"", ""description"": ""Adventure""}, {""i...",0,0,NaT,1
25615,game,Her,889680,0,0,None